In [ ]:
from sklearn.datasets import load_boston
import os



import pathlib

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import os
# boston = load_boston()

In [ ]:
import pandas as pd



column_names = ['fileId', 'row','rising_idx','falling_idx','volume','rising_weight',
                'falling_weight', 'delay']

raw_dataset = pd.read_csv(os.getenv('DATA_FS') + '../preprocessing/processed.csv', names=column_names,
                      na_values = "?", comment='\t',
                      sep=",", skipinitialspace=True, usecols = column_names[2:])

dataset = raw_dataset.copy()
dataset.tail()

dataset = dataset.dropna()
dataset = pd.get_dummies(dataset, prefix='', prefix_sep='')
dataset.tail()
train_dataset = dataset.sample(frac=0.8,random_state=0)
test_dataset = dataset.drop(train_dataset.index)
sns.pairplot(train_dataset[["rising_idx", "falling_idx", "volume", "rising_weight", "falling_weight", "delay"]], diag_kind="kde")
train_stats = train_dataset.describe()
train_stats.pop("delay")
train_stats = train_stats.transpose()
train_stats
train_labels = train_dataset.pop('delay')
test_labels = test_dataset.pop('delay')


# data = pd.DataFrame(boston.data)
# data.columns = boston.feature_names

In [ ]:
dataset.head()

In [ ]:
# data['PRICE'] = boston.target

In [ ]:
dataset.info()

In [ ]:
dataset.describe

In [ ]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np


In [ ]:
X, y = dataset.iloc[:,:-1],dataset.iloc[:,-1]

In [ ]:
data_dmatrix = xgb.DMatrix(data=X,label=y)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [ ]:
NUM_TREES = 10
MAX_DEPTH = 5
BASE_SCORE = 0.5

xg_reg = xgb.XGBRegressor(objective ='reg:linear', base_score = BASE_SCORE, 
                          colsample_bytree = 0.3, learning_rate = 0.1,
                          max_depth = MAX_DEPTH, alpha = 10, n_estimators = NUM_TREES)

In [ ]:
xg_reg.fit(X_train,y_train)

preds = xg_reg.predict(X_test)

In [ ]:
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

In [ ]:
params = {"objective":"reg:linear",'colsample_bytree': 0.3,'learning_rate': 0.1,
                'max_depth': 6, 'alpha': 10}

cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=3,
                    num_boost_round=50,early_stopping_rounds=20,metrics="rmse", as_pandas=True, seed=123)

In [ ]:
cv_results.tail()

In [ ]:
print((cv_results["test-rmse-mean"]).tail(1))

In [ ]:
xg_reg = xgb.train(params=params, dtrain=data_dmatrix, num_boost_round=10)

In [ ]:
import matplotlib.pyplot as plt

import matplotlib as mpl
mpl.rcParams['figure.dpi']= 400

xgb.plot_tree(xg_reg,num_trees=0)
plt.rcParams['figure.figsize'] = [50, 10]
plt.show()

In [ ]:
xgb.plot_importance(xg_reg)
plt.rcParams['figure.figsize'] = [50,10]
plt.show()

In [ ]:
df2 = pd.DataFrame({"CRIM":[0],
"ZN":[0],
"INDUS":[0],
"CHAS":[0],
"NOX":[0],
"RM":[0],
"AGE":[0],
"DIS":[0],
"RAD":[0],
"TAX":[0],
"PTRATIO":[0],
"B":[0],
"LSTAT":[0]}
) 
print(xg_reg.predict(df2))

In [ ]:
import pickle
import json
import jsonpickle
xg_reg.get_booster().dump_model('example_params')
# xg_reg.get_booster().get_dump()
numlines = len(open('example_params').readlines(  ))

import numpy
import math
fields = {}
threshes = {}
lefts = {}
rights = {}
tree = -1
num_nodes = int(math.pow(2,MAX_DEPTH+1))

with open('example_params') as file:
    for cnt, line in enumerate(file):
        if (not 'booster' in line):
            idx = int(line.strip().split(':')[0])
            if ('leaf=' in line):
                thresh = float(line.strip().split('leaf=')[-1])
                fields[idx] = ''
                threshes[idx] = thresh
            else:
                fields[idx] = line.strip().split('[')[1].split('<')[0]
                threshes[idx] = float(line.strip().split('[')[1].split('<')[1].split(']')[0])
                lefts[idx] = int(line.strip().split('yes=')[1].split(',')[0])
                rights[idx] = int(line.strip().split('no=')[1].split(',')[0])
        if (('booster' in line and cnt != 0) or cnt == numlines-1):
            tree = tree + 1
            scala = ['("", 0.to[T])' for i in range(num_nodes)]
    
            # Decompress the tree into a complete tree
            leftsArr = [lefts[0]]
            rightsArr = [rights[0]]
            scala[0] = '("%s", %f.to[T])' % (fields[0], threshes[0])
            for idx in range(1,num_nodes):
                if (idx % 2 == 1 and leftsArr[int(idx/2)] != None):
                    lookup = leftsArr[int(idx/2)]
                elif (idx % 2 == 0 and rightsArr[int(idx/2)-1] != None):
                    lookup = rightsArr[int(idx/2)-1]
                else: lookup = None
                
                if (lookup != None):
                    scala[idx] = '("%s", %f.to[T])' % (fields[lookup], threshes[lookup])
                    if (lookup in lefts):
                        leftsArr.append(lefts[lookup])
                    else:
                        leftsArr.append(None)
                    if (lookup in rights):
                        rightsArr.append(rights[lookup])
                    else:
                        rightsArr.append(None)
                else:
                    leftsArr.append(None)
                    rightsArr.append(None)


            print('val tree%d = List(' % tree + ','.join(scala).replace(' ','') + ')')
            fields = {}
            threshes = {}
            lefts = {}
            rights = {}
        
print('val trees = List(%s)' % ','.join(['tree%d' % d for d in range(tree+1)]))
print('val bias_score = %f' % BASE_SCORE)
#         print("Line {}: {}".format(cnt, line))
